### Reading in the file
Importing required libraries, connecting to the drive, and reading the dataset

In [1]:
import os
import re
import numpy as np
import pandas as pd
# from google.colab import drive
import random
# drive.mount('/content/drive')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [2]:
# df=pd.read_csv("/content/drive/My Drive/ESCP/ML project/train.csv")
df=pd.read_csv("/Users/andreasabia/Documents/ESCP/Machine Learning With Python/project/train.csv")

## Data Cleaning

In [3]:
# Dropping all the missing values from the keywords colum
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

# storing id,sex and gender
target=df.drop('keywords', axis=1)

#turning every keyword into a string
df=df.applymap(str)

# instantiating an empty dataframe
output=pd.DataFrame()

#getting the df ready for preprocessing
df.drop(['age','sex','ID'], axis=1, inplace=True)

#Introducing variables and datasets for data cleaning
lexicon=pd.read_csv("/Users/andreasabia/Downloads/Lexique383/Lexique383.tsv", sep='\t')
stop_words = stopwords.words('french')
stemmer = SnowballStemmer('french')
fr_dict=lexicon.ortho.tolist()

In [4]:
#looping over the first 20000 records, loading them in batches
for j in range(0,20000,100):

        #  selecting a subset of the data
        data=df[j:j+100]
    
        ### expanding the keywords into multiple columns
        data=data['keywords'].str.get_dummies(sep=';')

        
        ### Deleting all the columns which are made of digits, or do not seem to make any sense
        oldcols= data.columns.tolist()
        newcols=[]
        for i in range(len(oldcols)):
            if bool(re.search(r'\d', oldcols[i].split(':',1)[0]))==True or \
            bool(re.search(r'\.',oldcols[i].split(':',1)[0]))==True or \
            bool(re.search(r'\+',oldcols[i].split(':',1)[0]))==True or \
            bool(re.search(r'\_',oldcols[i].split(':',1)[0]))==True or \
            oldcols[i].split(':',1)[0] in stop_words:
                newcols.append('number')
            else:
                newcols.append(oldcols[i])

        data.set_axis(newcols, axis=1, inplace=True)
        data.drop(['number'], axis=1, inplace=True)

        ### Inputting the value at the end of the string as the value in the column
        for c,r in data.iterrows():
            for i in range(len(data.columns)):
                if r[i] != 0:
                    r[i]=data.columns[i].split(':',1)[1]

        ### Merging the columns which have the same keyword, by keeping the max for every record
        oldcols=data.columns.tolist()
        newcols=[]

        for i in range(len(oldcols)):
            newcols.append(oldcols[i].split(':',1)[0])
        data.set_axis(newcols, axis=1, inplace=True)
        
        oldcols=data.columns.tolist()
        newcols=[]

        for i in newcols:
            if i in fr_dict :
                newcols.append(stemmer.stem(i))
            else:
                newcols.append(i)

        data=data.replace(0,np.NaN)
        
        keyword=data.groupby(level=0, axis=1).max()

        ### merging the various chunks

        output=output.append(keyword, ignore_index=True, sort=True)


In [6]:
# Deleting all columns that only have 1 instances of the searched keyword
instances=output.count().tolist()

for i in range(len(instances)):
    if instances[i]==1:
        instances[i]=1
    else:
        instances[i]=0

colstodelete=[]
for i in range(len(output.columns)):
    if instances[i] == 1:
        colstodelete.append(output.columns[i])

output.drop(colstodelete, axis=1, inplace=True)

In [8]:
cleandata=output.merge(target, how='inner', left_index=True, right_index=True)

In [ ]:
cleandata.drop('index_x', axis=1, inplace=True)

In [ ]:
# merging the dataset with the targets
cleandata=pd.concat([target,output], axis=1)

In [11]:
#saving the clean dataset
cleandata.to_csv('clean_train3.csv', sep=';')